# Introduction to Automation with LangChain, Generative AI, and Python
**Lab 4: Add Pictures to Book**
* Instructor: [Jeff Heaton](https://youtube.com/@HeatonResearch), WUSTL Center for Analytics and Business Insight (CABI), [Washington University in St. Louis](https://olin.wustl.edu/faculty-and-research/research-centers/center-for-analytics-and-business-insight/index.php)
* For more information visit the [class website](https://github.com/jeffheaton/cabi_genai_automation).

# Instructions

Recall the book generator that we saw earlier? Lets add a cover picture to the title page. Below is a function that will render an image from the specified prompt and output to a filename. This is the same code that was presented earlier.

In [ ]:
import base64
import io
import json
import os
import sys

import boto3
from PIL import Image
import botocore

def render_diffusion(prompt, filename, height, width):
    boto3_bedrock = boto3.client('bedrock-runtime')
    negative_prompts = [
        "poorly rendered",
        "poor background details",
        "writing",
        "text",
    ]
    style_preset = "photographic"
    clip_guidance_preset = "FAST_GREEN"
    sampler = "K_DPMPP_2S_ANCESTRAL"

    request = json.dumps({
        "text_prompts": (
            [{"text": prompt, "weight": 1.0}]
            + [{"text": negprompt, "weight": -1.0} for negprompt in negative_prompts]
        ),
        "cfg_scale": 5,
        "seed": 42,
        "steps": 60,
        "style_preset": style_preset,
        "clip_guidance_preset": clip_guidance_preset,
        "sampler": sampler,
        "width": width,
        "height": height
    })
    modelId = "stability.stable-diffusion-xl-v1"

    response = boto3_bedrock.invoke_model(body=request, modelId=modelId)
    response_body = json.loads(response.get("body").read())

    print(response_body["result"])
    base_64_img_str = response_body["artifacts"][0].get("base64")
    image = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
    image.save(filename)
    return image


Provided here is the book code from before. Modify this code to add an image that you generate with the render_diffusion function. Claude 3 can give you the code to add an image to the markdown, but it may not be able to make the complete change. You should also call the LLM to transform the synopsis into an image generation prompt.

In [4]:
# Imports organized by source and purpose
from IPython.display import display_markdown
import markdown
import pdfkit

from langchain import OpenAI, PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_aws import ChatBedrock

# Constants used in the program
MODEL = 'anthropic.claude-3-sonnet-20240229-v1:0'
SUBJECT = "international spy story"

# Initialize ChatBedrock with specified model and settings
llm = ChatBedrock(
    model_id=MODEL,
    model_kwargs={"temperature": 0.7}
)

def query_llm(prompt):
    """Query the language model with a formatted prompt and return the response content."""
    messages = [
        SystemMessage(
            content="""You are writing a book, return just what I ask you for. 
                       Do not add comments, it is very important that you only 
                       provide the final output without any additional comments 
                       or remarks. Do not repeat my command or a summary of my command."""
        ),
        HumanMessage(content=prompt)
    ]

    output = llm.invoke(messages)
    return output.content.strip(" '\"")

# Generate book title, synopsis, and table of contents
title = query_llm(f"Give me a random title for a book on the subject '{SUBJECT}'. Return only the title, no additional text.")
print(f"Title: {title}")

synopsis = query_llm(f"Give me just a synopsis for a book of the title '{title}' on the subject '{SUBJECT}'.")
print(f"Synopsis: {synopsis}")


toc = query_llm(f"Give me a table of contents for a book of the title '{title}' on the subject '{SUBJECT}' "
                f"the book synopsis is '{synopsis}'. Return the table of contents as a list of chapter titles. "
                f"Separate the chapter number and chapter title with a pipe character '|'. "
                f"Do not give me any line that is not a chapter!")

# Split the table of contents into lines and extract chapter titles
lines = toc.splitlines()
toc2 = [line.split('|')[1].strip() for line in lines if line]

# Render the book chapters
book = f"# {title}\n{synopsis}\n\n## Table of Contents\n\n" + "\n".join(f"{i+1}. {title}" for i, title in enumerate(toc2))

def render_chapter(num, chapter_title):
    """Generate content for a chapter given the chapter number and title."""
    txt = query_llm(f"Write Chapter {num}, titled '{chapter_title}' for a book of the title '{title}' on "
                    f"the subject '{SUBJECT}' the book synopsis is '{synopsis}' the table of contents is '{toc}'. "
                    f"Give me only the chapter text, no chapter heading, no chapter title, number, no additional text. "
                    f"Make sure the chapter is at least 3,000 to 4,000 words. End with a complete sentence, do not cut off.")
    return txt

# Render the entire book
for chapter_num, chapter_title in enumerate(toc2, start=1):
    print(f"Rendering chapter {chapter_num}/{len(toc2)}: {chapter_title}")
    chapter_text = render_chapter(chapter_num, chapter_title)
    book += f"\n\n## Chapter {chapter_num}: {chapter_title}\n{chapter_text}"

# Convert Markdown to HTML and then to PDF
html = markdown.markdown(book)
pdfkit.from_string(html, 'output.pdf', options={"enable-local-file-access": ""})


Title: Collateral Deception
Synopsis: A high-stakes international spy thriller, 'Collateral Deception' follows the exploits of CIA operative Sarah Knowles as she unravels a web of deceit and betrayal that threatens global security. After a routine mission goes awry, Sarah finds herself embroiled in a deadly conspiracy involving rogue agents, powerful corporations, and a sinister plot that could ignite a new world war. With the fate of nations hanging in the balance, Sarah must navigate a treacherous landscape of lies and double-crosses, relying on her wits and skills to uncover the truth before it's too late.
Rendering chapter 1/13: Prologue: The Calm Before the Storm
Rendering chapter 2/13: Whispers in the Night
Rendering chapter 3/13: Echoes of Betrayal
Rendering chapter 4/13: Shadows in the Boardroom
Rendering chapter 5/13: The Rabbit Hole
Rendering chapter 6/13: Ghosts of the Past
Rendering chapter 7/13: The Moscow Connection
Rendering chapter 8/13: Infiltration
Rendering chapter 9

True